# **FreshLens AI: Klasifikasi Buah dengan MobileNetV2**
FreshLens AI adalah sistem klasifikasi citra cerdas berbasis Deep Learning yang dirancang untuk mengenali jenis buah secara real-time dengan presisi tinggi. Menggunakan arsitektur MobileNetV2, sistem ini mampu memproses input visual dan memberikan identifikasi instan (Apel, Pisang, Salak) melalui antarmuka dashboard yang interaktif dan modern.

---

## **1. Otomatisasi Akuisisi dan Kurasi Dataset**
Modul ini berfungsi sebagai pipeline persiapan data otomatis yang menghubungkan lingkungan kerja (Google Colab) dengan repositori Kaggle melalui otentikasi API. Kode ini mengelola siklus hidup data mulai dari pengunduhan (downloading) dataset Fruits-360, ekstraksi arsip ZIP, hingga proses data curating—yaitu memfilter dan hanya menyalin folder kelas target yang relevan (Apel, Pisang, Salak) ke dalam struktur direktori baru, sehingga menghasilkan dataset yang bersih, ringan, dan siap digunakan langsung untuk pelatihan model tanpa intervensi manual.

---

In [11]:
import os
import shutil
import zipfile
import json

# --- 1. SETTING AKUN KAGGLE (Otomatis) ---
KAGGLE_USERNAME = "PAKAI USERNAME KAGGLE ANDA"
KAGGLE_KEY = "PAKAI KEY API KAGGLE ANDA"

token = {"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}
!mkdir -p ~/.kaggle
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 ~/.kaggle/kaggle.json

# --- 2. DOWNLOAD DATASET FRUITS-360 (Versi Ringan) ---
print("🚀 Sedang mendownload dataset Fruits-360...")
print("⏳ Tunggu sebentar ya...!")

!pip install -q kaggle
!kaggle datasets download -d moltean/fruits --force

# --- 3. EKSTRAK DATA ---
print("📦 Sedang membuka paket data (Unzip)...")
with zipfile.ZipFile("fruits.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/raw_dataset")

# --- 4. SIAPKAN DATA SPESIFIK (Hanya Ambil 3 Buah) ---
print("🧹 Sedang memilah hanya buah Apel, Pisang, dan Salak...")

# Folder asli hasil ekstrak (Struktur Fruits-360)
source_dir = '/content/raw_dataset/fruits-360_dataset/fruits-360/Training'

# Folder tujuan
base_dir = '/content/dataset_siap_latih'
if os.path.exists(base_dir): shutil.rmtree(base_dir)
os.makedirs(base_dir)

# Kita pilih varian buah yang paling umum
target_map = {
    'Apel': 'Apple Red 1',
    'Pisang': 'Banana 1',
    'Salak': 'Salak 1'
}

berhasil = 0
for nama_kita, nama_asli in target_map.items():
    src = os.path.join(source_dir, nama_asli)
    dst = os.path.join(base_dir, nama_kita)

    if os.path.exists(src):
        shutil.copytree(src, dst)
        print(f"   ✅ {nama_kita} berhasil disiapkan!")
        berhasil += 1
    else:
        src_alt = os.path.join('/content/raw_dataset/fruits-360/Training', nama_asli)
        if os.path.exists(src_alt):
            shutil.copytree(src_alt, dst)
            print(f"   ✅ {nama_kita} berhasil disiapkan! (Path Alt)")
            berhasil += 1
        else:
            print(f"   ❌ Gagal menemukan: {nama_asli}")

if berhasil == 3:
    print(f"\n🎉 SUKSES! Dataset siap di: {base_dir}")
    print("Sekarang dataset kamu enteng banget, training pasti ngebut!")
else:
    print("\n⚠️ Ada folder yang tidak ketemu, coba cek log di atas.")

🚀 Sedang mendownload dataset Fruits-360...
⏳ Tunggu sebentar ya...!
Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
^C
📦 Sedang membuka paket data (Unzip)...
🧹 Sedang memilah hanya buah Apel, Pisang, dan Salak...
   ❌ Gagal menemukan: Apple Red 1
   ❌ Gagal menemukan: Banana 1
   ❌ Gagal menemukan: Salak 1

⚠️ Ada folder yang tidak ketemu, coba cek log di atas.


## **2. Algoritma Pencarian Direktori Cerdas dan Penyiapan Dataset Hirarkis**
Kode ini mengimplementasikan logika pencarian direktori rekursif (recursive directory walking) menggunakan os.walk untuk menemukan lokasi folder "Training" secara dinamis tanpa bergantung pada struktur path yang kaku. Setelah lokasi ditemukan, skrip melakukan kurasi data adaptif dengan memetakan variasi nama folder kelas target (misalnya mendeteksi antara "Banana" atau "Banana 1") dan menyalin data yang valid ke direktori kerja baru, memastikan dataset akhir terstruktur rapi dan bebas dari kesalahan jalur (path errors) sebelum proses pelatihan model.

---

In [12]:
import os
import shutil

print("🕵️‍♂️ Sedang mencari lokasi folder 'Training' yang asli...")

# 1. CARI FOLDER 'Training' SECARA OTOMATIS
# Kita tidak mau menebak path-nya, biarkan Python yang mencarinya.
base_search_dir = '/content/raw_dataset'
training_dir = None

for root, dirs, files in os.walk(base_search_dir):
    if 'Training' in dirs:
        training_dir = os.path.join(root, 'Training')
        print(f"✅ KETEMU! Folder Training ada di: {training_dir}")
        break

if not training_dir:
    print("❌ Gawat, folder 'Training' tidak ketemu sama sekali. Cek tab Files di kiri.")
else:
    # 2. SIAPKAN FOLDER TUJUAN
    base_dir = '/content/dataset_siap_latih'
    if os.path.exists(base_dir): shutil.rmtree(base_dir)
    os.makedirs(base_dir)

    # 3. DAFTAR BUAH YANG MAU DIAMBIL
    # Nama Key (Kiri): Nama folder yang akan kita buat
    # Nama Value (Kanan): Nama folder ASLI di dataset Kaggle (Harus persis)

    # Tips: Di Fruits-360, biasanya namanya 'Banana' (bukan Banana 1) dan 'Salak' (bukan Salak 1)
    # Saya buat list alternatif biar dia cari mana yang ada.
    target_map = {
        'Apel': ['Apple Red 1', 'Apple Red'],
        'Pisang': ['Banana', 'Banana 1'],
        'Salak': ['Salak', 'Salak 1']
    }

    print("\n📦 Mulai menyalin data...")

    total_sukses = 0

    for nama_kita, variasi_nama_asli in target_map.items():
        found_fruit = False

        # Cek setiap variasi nama (misal cek 'Banana' dulu, kalau gak ada cek 'Banana 1')
        for nama_asli in variasi_nama_asli:
            src = os.path.join(training_dir, nama_asli)

            if os.path.exists(src):
                dst = os.path.join(base_dir, nama_kita)
                shutil.copytree(src, dst)
                print(f"   ✅ Sukses menyalin: {nama_asli} -> ke folder {nama_kita}")
                found_fruit = True
                total_sukses += 1
                break # Kalau sudah ketemu satu, stop cari variasi lain

        if not found_fruit:
            print(f"   ❌ Gagal menemukan buah: {nama_kita} (Mencoba cari: {variasi_nama_asli})")

    # 4. HASIL AKHIR
    if total_sukses == 3:
        print(f"\n🎉 SEMPURNA! 3 Buah sudah siap di: {base_dir}")
        print("Cek folder 'dataset_siap_latih' di menu kiri, isinya pasti ada Apel, Pisang, Salak.")
    else:
        print("\n⚠️ Ada buah yang belum ketemu. Coba cek daftar nama folder di: " + training_dir)
        # Tampilkan isi folder Training biar kamu bisa cek nama aslinya apa
        print("Isi folder Training (10 teratas):", os.listdir(training_dir)[:10])

🕵️‍♂️ Sedang mencari lokasi folder 'Training' yang asli...
✅ KETEMU! Folder Training ada di: /content/raw_dataset/fruits-360_100x100/fruits-360/Training

📦 Mulai menyalin data...
   ✅ Sukses menyalin: Apple Red 1 -> ke folder Apel
   ✅ Sukses menyalin: Banana 1 -> ke folder Pisang
   ✅ Sukses menyalin: Salak 1 -> ke folder Salak

🎉 SEMPURNA! 3 Buah sudah siap di: /content/dataset_siap_latih
Cek folder 'dataset_siap_latih' di menu kiri, isinya pasti ada Apel, Pisang, Salak.


## **3. Pipeline Pelatihan Transfer Learning Adaptif dengan Arsitektur MobileNetV2**
Skrip ini mengorkestrasi proses pelatihan model Deep Learning secara end-to-end, dimulai dengan pra-pemrosesan citra melalui teknik Data Augmentation (rotasi, pergeseran, dan flip) untuk mencegah overfitting. Inti dari sistem ini menerapkan teknik Transfer Learning menggunakan arsitektur MobileNetV2 sebagai pengekstraksi fitur (feature extractor) yang efisien, dilengkapi dengan lapisan klasifikasi kustom (custom head) yang dirancang secara dinamis agar otomatis menyesuaikan jumlah output neuron dengan jumlah kelas dalam dataset, serta diakhiri dengan mekanisme ekspor model (.h5) yang siap untuk tahap deployment.

---

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import os

# --- 1. KONFIGURASI ---
DATA_DIR = '/content/dataset_siap_latih'
IMG_SIZE = (224, 224)  # Ukuran standar MobileNet
BATCH_SIZE = 32
EPOCHS = 5             # Cukup 5 putaran karena datanya bersih & modelnya pintar

# --- 2. PERSIAPAN DATA (AUGMENTASI) ---
# Teknik ini memutar/menggeser gambar sedikit biar AI lebih jago
train_datagen = ImageDataGenerator(
    rescale=1./255,        # Normalisasi (wajib)
    rotation_range=20,     # Putar-putar dikit
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,  # Balik kiri-kanan
    validation_split=0.2   # 20% data diambil buat ujian (Tes)
)

print("📚 Menyiapkan Data Latih...")
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

print("📝 Menyiapkan Data Validasi (Ujian)...")
validation_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Cek urutan label (PENTING BUAT FLASK NANTI)
labels = (train_generator.class_indices)
print(f"\n🏷️ Urutan Kelas AI: {labels}")
# Hasilnya biasanya: {'Apel': 0, 'Pisang': 1, 'Salak': 2}

# --- 3. BANGUN MODEL (MobileNetV2) ---

# 1. HITUNG JUMLAH KELAS OTOMATIS
jumlah_kelas = len(train_generator.class_indices)
print(f"🤖 Model akan dibuat untuk mendeteksi {jumlah_kelas} jenis buah.")

# Download otak MobileNet tanpa "kepala"
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Bekukan base_model
base_model.trainable = False

# Pasang "kepala" baru
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)

# --- BAGIAN PENTING YANG DIUBAH ---
# Jangan tulis angka 3, tapi pakai variabel 'jumlah_kelas'
predictions = Dense(jumlah_kelas, activation='softmax')(x)
# ----------------------------------

model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --- 4. MULAI TRAINING ---
print(f"\n🚀 Mulai Melatih AI selama {EPOCHS} putaran... (Tunggu sebentar)")
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    verbose=1
)

# --- 5. SIMPAN & DOWNLOAD ---
nama_file = 'model_buah.h5'
model.save(nama_file)
print(f"\n🎉 TRAINING SELESAI! Model disimpan sebagai: {nama_file}")

# Kode download otomatis ke laptop
try:
    from google.colab import files
    files.download(nama_file)
    print("⬇️ Sedang mendownload ke laptop kamu...")
except:
    print("⚠️ Download manual: Cek ikon folder di kiri -> klik kanan 'model_buah.h5' -> Download")

📚 Menyiapkan Data Latih...
Found 1178 images belonging to 3 classes.
📝 Menyiapkan Data Validasi (Ujian)...
Found 294 images belonging to 3 classes.

🏷️ Urutan Kelas AI: {'Apel': 0, 'Pisang': 1, 'Salak': 2}
🤖 Model akan dibuat untuk mendeteksi 3 jenis buah.

🚀 Mulai Melatih AI selama 5 putaran... (Tunggu sebentar)
Epoch 1/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.8985 - loss: 0.2360 - val_accuracy: 1.0000 - val_loss: 0.0082
Epoch 2/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 1.0000 - loss: 4.1914e-04 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 3/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 1.0000 - loss: 3.5725e-04 - val_accuracy: 1.0000 - val_loss: 8.8775e-04
Epoch 4/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 1.0000 - loss: 1.8556e-04 - val_accuracy: 1.0000 - val_loss: 6.3668e-04
Epoch 5/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 1.0000 - loss: 1.3464e-04 - val_accuracy: 1.0000 - val_loss: 7.8681e-04



🎉 TRAINING SELESAI! Model disimpan sebagai: model_buah.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Sedang mendownload ke laptop kamu...
